## 任务背景

这个比赛的任务就是**文本分类**，是自然语言处理(NLP)领域里一项最最基本的任务。但这个任务的难点就是在于， 文本的长度非常长，大约3000个词，一般任务也就300词。而文本的长度过长对文本的智能解析带来了很多挑战。

用传统的监督学习模型对一段文本进行分类的基本过程:

一段原始文本通过数据预处理得到处理后的文本，然后进行特征工程，得到Features，之后输入模型$y = f(x_1,x_2,...,x_n)$，得到输出类别。

In [1]:
import pandas as pd

In [2]:
# 导入训练和测试数据集
df_train = pd.read_csv('../data/train_set.csv')
df_test = pd.read_csv('../data/test_set.csv')

In [3]:
df_train.shape

(102277, 4)

In [4]:
df_train.head()

,id,article,word_seg,class
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12


可以看出，训练数据集包括id，article，word_seg和class，总计102277行数据。

数据中的数字都代表的是脱敏后的字或者词。article是一篇由字组成的文章，word_seg是由词组成的文章，class代表对应的类别。

## 第一步——数据预处理

In [5]:
df_train.describe()

,id,class
count,102277.000000,102277.000000
mean,51138.000000,10.262356
std,29524.971078,5.370785
min,0.000000,1.000000
25%,25569.000000,6.000000
50%,51138.000000,10.000000
75%,76707.000000,15.000000
max,102276.000000,19.000000


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102277 entries, 0 to 102276
Data columns (total 4 columns):
id          102277 non-null int64
article     102277 non-null object
word_seg    102277 non-null object
class       102277 non-null int64
dtypes: int64(2), object(2)
memory usage: 3.1+ MB


分析比赛数据可以看出数据非常的干净，没有空缺值等，只需要直接上手即可。

In [7]:
df_train.drop(columns=['article', 'id'], inplace=True)
df_test.drop(columns=['article'], inplace=True)

使用pandas读入数据后， 删除了article和id这两列不需要的信息。

article是由字组成的，单个字很多是不存在意义的，会对模型产生误导信息，简单的不进行处理，直接删除。

## 第二步——特征工程

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.9, max_features=100000)
vectorizer.fit(df_train['word_seg'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=100000, min_df=3,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

这里使用scikit-learn中的CountVectorizer从文章中提取出特征信息，CountVectorizer只考虑词汇在文本中出现的频率。

除此之外，还可以使用TfidfVectorizer，除了考量某一词汇在当前训练文本中出现的频率之外，同时关注包含这个词汇的其它训练文本数目的倒数，能够削减高频没有意义的词汇出现带来的影响, 挖掘更有意义的特征。

在这个阶段，将字符文本转化成数字向量，以便计算机能够进行处理。

In [10]:
x_train = vectorizer.transform(df_train['word_seg'])
x_test = vectorizer.transform(df_test['word_seg'])
y_train = df_train['class'] - 1

## 第三步——构建模型进行训练

### 逻辑回归方案

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
lg = LogisticRegression(C=4, dual=True)
lg.fit(x_train, y_train)

/Users/ryne/Library/Python/3.6/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/ryne/Library/Python/3.6/lib/python/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/ryne/Library/Python/3.6/lib/python/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

模型中的c指定了惩罚函数的倒数，值越小，正则化越大。此外，模型还有一些其他的参数:penalty指定正则化策略，solver求解最优化问题的算法等，

## 第四步——输出

In [13]:
y_test = lg.predict(x_test)

In [14]:
df_test['class'] = y_test.tolist()
df_test['class'] = df_test['class'] + 1
df_result = df_test.loc[:, ['id', 'class']]

In [16]:
df_result.to_csv('../data/result.csv', index=False)